# Important clear all outputs before saving, otherwise its around 500 mb

In [ ]:
import glob
import sys
import warnings

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import SubplotSpec
from skimage import feature, measure
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from skimage.morphology import dilation, closing, white_tophat, disk
from sklearn.metrics.pairwise import cosine_similarity as cos_sim

from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 2**128
warnings.filterwarnings("ignore",category=UserWarning)



In [ ]:
frames = 50
frame_steps = 5
eval_type = "test"
dataset = f"road_tree_new_{eval_type}" # road_tree_new_test or road_tree_new_train
real_style_0 = glob.glob(f'./styletransfer/cut/datasets/{dataset}/trainB/*_0.jpg*')
real_style_1 = glob.glob(f'./styletransfer/cut/datasets/{dataset}/trainB/*_1.jpg*')
real_style_0 = np.array([np.asarray(Image.open(fname)) for fname in real_style_0])
real_style_1 = np.array([np.asarray(Image.open(fname)) for fname in real_style_1])

In [ ]:
loaded = np.load(f"files_for_style_comparisson/continuous_trees_test.npz")#np.load(f"files_for_style_comparisson/new_trees_{eval_type}.npz")#np.load("files_for_style_comparisson/createdat_2021-06-13 08:46:46.597627.npz")
names = loaded["names"]
steps = loaded["steps"]
experiments = loaded["frames"]

openAI_data = experiments[0]

edges = loaded['edges']
edges = np.squeeze(edges/255).astype(float)
edge_names = loaded['edge_names']

# FID score

In [ ]:
import os
import shutil
from pytorch_fid.fid_score import calculate_fid_given_paths
from glob import glob


In [ ]:
def FID_score(names,experiments,real_style_0,real_style_1, dim):
    tmp_path = './tmp'
    os.mkdir(tmp_path)
    tgt_path0=tmp_path+'/target0'
    os.mkdir(tgt_path0)
    tgt_path1=tmp_path+'/target1'
    os.mkdir(tgt_path1)
    pred_path = tmp_path+'/prediction'
    os.mkdir(pred_path)
    nms = [n[10:] for n in names[1:]]
    predictions =experiments[1:]

    for i, image in enumerate(real_style_0):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f"{tgt_path0}/tgt_{i}.jpg", image)

    for i, image in enumerate(real_style_1):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f"{tgt_path1}/tgt_{i}.jpg", image)

    for j,exp in enumerate(predictions):
        os.mkdir(f"{pred_path}/{nms[j]}")
        for i, image in enumerate(exp):
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(f"{pred_path}/{nms[j]}/pred_{i}.jpg", image)

    os.mkdir(f"{pred_path}/org")
    for i, image in enumerate(experiments[0]):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f"{pred_path}/org/pred_{i}.jpg", image)

    dev = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')


    fid_values = [calculate_fid_given_paths(paths=[tgt, pred], batch_size=50, device=dev, dims=dim) for (tgt, pred) in zip([tgt_path0, tgt_path1]*((len(nms)//2)+3),[tgt_path0, tgt_path1]+[f"{pred_path}/{nm}" for nm in nms]+[f"{pred_path}/org"]*2)]

    shutil.rmtree(tmp_path)
    labels = ['originals']+[n[14:-2] for n in nms[::2]]+["openAI"]
    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(20,5))
    rects1 = ax.bar(x - width/2, fid_values[::2], width, label='Style 0', color='yellowgreen')
    rects2 = ax.bar(x + width/2, fid_values[1::2], width, label='Style 1', color='saddlebrown')
    ax.set_ylim(top= np.max(fid_values[:-2])*1.1)


    if dim == 64: 
        conf = "first max pooling features"
    if dim == 192: 
        conf = "second max pooling features"
    if dim == 768: 
        conf = "pre-aux classifier features"
    if dim == 2048:
        conf = "final average pooling features"


    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('FID Scores')
    ax.set_title(f"FID Score per Model and style comparing predictions and target datasets(lower is better)\nfeature dim {dim}\n {conf}; OpenAI vals {fid_values[-2]:.1f}, {fid_values[-1]:.1f}")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()



    fig.tight_layout()
    fig.savefig(f'Slides/FID_{eval_type}_dim_{dim}_scaled.png', dpi=300)
    plt.show()
    
    fig, ax = plt.subplots(figsize=(20,5))
    rects1 = ax.bar(x - width/2, fid_values[::2], width, label='Style 0', color='yellowgreen')
    rects2 = ax.bar(x + width/2, fid_values[1::2], width, label='Style 1', color='saddlebrown')


    if dim == 64: 
        conf = "first max pooling features"
    if dim == 192: 
        conf = "second max pooling features"
    if dim == 768: 
        conf = "pre-aux classifier features"
    if dim == 2048:
        conf = "final average pooling features"


    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('FID Scores')
    ax.set_title(f"FID Score per Model and style comparing predictions and target datasets(lower is better)\nfeature dim {dim}\n {conf}")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()



    fig.tight_layout()
    fig.savefig(f'Slides/FID_{eval_type}_dim_{dim}.png', dpi=300)
    plt.show()

In [ ]:
FID_score(names,experiments,real_style_0,real_style_1, 2048)

In [ ]:
FID_score(names,experiments,real_style_0,real_style_1, 768)

In [ ]:
FID_score(names,experiments,real_style_0,real_style_1, 768)

In [ ]:
FID_score(names,experiments,real_style_0,real_style_1, 192)

In [ ]:
FID_score(names,experiments,real_style_0,real_style_1, 64)

# Ploting functions

In [ ]:
#Existing functions
def do_canny_single(img, label, img_type):
    if img_type == 'real':
        img_gray = rgb2gray(img)
        if label == 1:
            # brown
            img_gray[img_gray <= 0.5] = 0
            img_gray[img_gray > 0.5] = 1
        else:
            # grey
            img_gray = feature.canny(img_gray, sigma=6, low_threshold=0.1)

        edges = img_gray  # pointer magic
    else:
        img_gray = np.squeeze(seg_sim(img))
        edges = feature.canny(img_gray, sigma=3, low_threshold=0.2)
    return edges
def seg_sim(image):
    aux2 = np.zeros((256, 256, 1), dtype=np.uint8)
    aux2[np.where((image[:256, :, 1] >= 98) & (image[:256, :, 1] <= 112))] = [255]
    aux2[np.where(image[:256, :, 1] == 107)] = [255]
    aux2[np.where(image[:256, :, 1] == 102)] = [255]
    aux2[np.where(image[:256, :, 1] == 111)] = [255]

    return aux2

In [ ]:
OpenAI_edges = np.array([do_canny_single(img, None, "OpenAI") for img in openAI_data]).astype(float)

In [ ]:
img_type = ["OpenAI"]+["real"]*14
style_class = [None]+[0,1]*7

In [ ]:
names

In [ ]:
generated_edges = [np.array([do_canny_single(img, style, typ) for img in gan_data]).astype(float) for (gan_data, style, typ) in zip(experiments,style_class, img_type)]

In [ ]:
gan_data_0_edges = np.array([do_canny_single(img, 0, "real") for img in gan_data_0]).astype(float)
gan_data_1_edges = np.array([do_canny_single(img, 1, "real") for img in gan_data_1]).astype(float)

In [ ]:
def get_losses(YH, y=OpenAI_edges):
    return [np.mean(YH==y), MSE(YH, y), BCE(YH, y), ssim(YH,y), psnr(y,YH), cos_sim(y, YH).mean()]

def MSE(YH, Y):
    loss = torch.nn.MSELoss()
    return loss(torch.from_numpy(Y), torch.from_numpy(YH)).numpy() 


def BCE(YH, Y):
    loss = torch.nn.BCELoss()
    return loss(torch.from_numpy(Y), torch.from_numpy(YH)).numpy()

In [ ]:

def histo_single_grayscale(image, title, ax):
    img = [image]
    img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in img])
    hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256])
    ax.cla()
    ax.set_title(f"Grayscale Histogram {title}")
    ax.set_xlabel("Bins")
    ax.set_ylabel("# of Pixels")
    ax.plot(hist)
    ax.set_xlim([0, 256])
    return ax


def histo_single(image, title, ax):
    img = [image]
    colors = ("r", "g", "b")
    ax.cla()
    ax.set_title(f"'Flattened' Color Histogram {title}")
    ax.set_xlabel("Bins")
    ax.set_ylabel("# of Pixels")
    for channel in range(3):
        hist = cv2.calcHist(img, [channel], None, [256], [0, 256])
        ax.plot(hist, color=colors[channel])
        ax.set_xlim([0, 256])
    return ax


def real_vs_gan_histo(real_data, gan_data, style, save=False):
    fig, ax = plt.subplots(2, 5, figsize=(30, 10))
    fig.patch.set_alpha(1.)

    n = 0
    fig.suptitle(f"Histogram comparison for style {style}, frame {n}")
    ax[0, 0].imshow(real_data[n])
    ax[0, 0].set_title('Input real image sample')
    ax[0, 0].set_axis_off()

    ax[0, 1] = histo_single_grayscale(real_data[n], f"single image Real", ax[0, 1])
    ax[0, 2] = histo_single_grayscale(real_data.reshape(real_data.shape[0] * real_data.shape[1], real_data.shape[2], real_data.shape[3]), f"mean Real", ax[0, 2])
    ax[0, 3] = histo_single(real_data[n], f"single image Real", ax[0, 3])
    ax[0, 4] = histo_single(real_data.reshape(real_data.shape[0] * real_data.shape[1], real_data.shape[2], real_data.shape[3]), f"mean Real", ax[0, 4])

    ax[1, 0].imshow(gan_data[n])
    ax[1, 0].set_title('Generated image sample')
    ax[1, 0].set_axis_off()

    ax[1, 1] = histo_single_grayscale(gan_data[n], f"single image Generated", ax[1, 1])
    ax[1, 2] = histo_single_grayscale(gan_data.reshape(gan_data.shape[0] * gan_data.shape[1], gan_data.shape[2], gan_data.shape[3]), f"mean Generated", ax[1, 2])
    ax[1, 3] = histo_single(gan_data[n], f"single image Generated", ax[1, 3])
    ax[1, 4] = histo_single(gan_data.reshape(gan_data.shape[0] * gan_data.shape[1], gan_data.shape[2], gan_data.shape[3]), f"mean Generated", ax[1, 4])
    plt.tight_layout()

    # plt.savefig(f"Slides/histo_style_{style}.png" , bbox_inches='tight')
    def update(n):
        fig.suptitle(f"Histogram comparison for style {style}, frame {n}")
        ax[0, 0].imshow(real_data[n])
        ax[0, 0].set_title('Input real image sample')
        ax[0, 0].set_axis_off()

        ax[0, 1] = histo_single_grayscale(real_data[n], f"single image Real", ax[0, 1])
        ax[0, 3] = histo_single(real_data[n], f"single image Real", ax[0, 3])

        ax[1, 0].imshow(gan_data[n])
        ax[1, 0].set_title('Generated image sample')
        ax[1, 0].set_axis_off()

        ax[1, 1] = histo_single_grayscale(gan_data[n], f"single image Generated", ax[1, 1])
        ax[1, 3] = histo_single(gan_data[n], f"single image Generated", ax[1, 3])

    anim = FuncAnimation(fig, update, frames=frames, interval=1000, save_count=sys.maxsize)
    if save:
        anim.save(f'Slides/histo_style_{style}.gif', dpi=150, writer='imagemagick')
        print(f'Slides/histo_style_{style}.gif done')

    plt.show()
    return anim

def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n')
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

def curve_detection(openAI_data, real_style_0, real_style_1, save=False):
    def h_trasn_simple(image, label, number, img_type, row):
        # Classic straight-line Hough transform

        edge = do_canny_single(image, label, img_type)

        # Generating figure 1

        ax[row, 0].imshow(image, cmap=cm.gray)
        ax[row, 0].set_title('Input image')
        ax[row, 0].set_axis_off()

        ax[row, 1].imshow(edge, cmap=cm.gray)
        ax[row, 1].set_title('Canny edge detection')
        ax[row, 1].set_axis_off()

        if img_type == "real":
            closed = closing(edge, disk(20))

            wt = white_tophat(closed, disk(1))
        else:
            wt = edge

        r = dilation(wt, disk(2))
        r = np.pad(r, 2)
        contours = measure.find_contours(r, 0.8)

        myline = np.linspace(0, 256, 100)

        ax[row, 2].cla()
        ax[row, 2].imshow(r, cmap=cm.gray)
        if img_type == "real":
            ax[row, 2].set_title('Processed (closing + white_tophat + dilation + padding + contours)')
        else:
            ax[row, 2].set_title('Processed (dilation + padding + contours)')
        ax[row, 2].set_axis_off()

        for contour in contours:
            ax[row, 2].plot(contour[:, 1], contour[:, 0], linewidth=2)

        ax[row, 3].cla()
        ax[row, 3].imshow(np.zeros(edge.shape), cmap=cm.gray)
        ax[row, 3].set_title('Fitted curve and points')
        ax[row, 3].set_axis_off()

        road_type = "Undefined"

        defined = True

        if len(contours) != 0:
            contours = contours[np.argmax([c.shape[0] for c in contours])]

            x = np.vstack(contours)[:, 0]
            y = np.vstack(contours)[:, 1]

            ax[row, 3].scatter(y, x)

            mymodel = np.poly1d(np.polyfit(x, y, 2))
            line = mymodel(myline)

            if mymodel(256) > 300 or mymodel(256) < -50:
                defined = False

            ax[row, 3].plot(line, myline, "--r")
        else:
            defined = False
            road_type = "No road"
        ax[row, 3].set_ylim((edge.shape[0], 0))
        ax[row, 3].set_xlim((0, edge.shape[1]))

        bottom = np.sum(edge[-20:, :])
        if bottom > 10 and defined:
            d2 = np.polyder(mymodel, m=2)
            d2 = d2(0) * 1000
            if d2 < -1:
                road_type = "left curve"
            elif d2 > 1:
                road_type = "right curve"
            else:
                road_type = "Straight road"

            road_type = f"{road_type}\nd2: {d2}"

        create_subtitle(fig, grid[row, ::], f"{road_type}")

    n = 0
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))
    fig.patch.set_alpha(1.)

    grid = plt.GridSpec(3, 4)

    h_trasn_simple(openAI_data[n], None, n, "OpenAI", 0)

    h_trasn_simple(real_style_0[n], 0, n, "real", 1)

    h_trasn_simple(real_style_1[n], 1, n, "real", 2)
    plt.tight_layout()

    def update(n):
        h_trasn_simple(openAI_data[n], None, n, "OpenAI", 0)

        h_trasn_simple(real_style_0[n], 0, n, "real", 1)

        h_trasn_simple(real_style_1[n], 1, n, "real", 2)

    anim = FuncAnimation(fig, update, frames=frames, interval=1000, save_count=sys.maxsize)
    if save:
        anim.save(f'Slides/curves.gif', dpi=150, writer='imagemagick')
        print(f'Slides/curves.gif done')
    return anim

In [ ]:
def edge_comp_graphs(experiments, OpenAI_edges, gan_data_0_edges, gan_data_1_edges, save=False):
    openAI_data, gan_data_0, gan_data_1 = experiments
    n = 0
    X = range(OpenAI_edges.shape[0])
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))
    fig.patch.set_alpha(1.)

    losses_0 = np.array([get_losses(y, x) for (x,y) in  zip(gan_data_0_edges,OpenAI_edges)])
    losses_1 = np.array([get_losses(y, x) for (x,y) in  zip(gan_data_1_edges,OpenAI_edges)])
    losses_n = ['Accuracy', 'MSE', 'BCE', 'SSIM', 'PSNR', 'Cosine similarity']
    
    
    def update(n):
        fig.suptitle(f"Structure comparison, frame {n}")
        ax[0, 0].imshow(openAI_data[n])
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()

        ax[1, 0].imshow(gan_data_0[n])
        ax[1, 0].set_title('Generated Style 0')
        ax[1, 0].set_axis_off()

        ax[2, 0].imshow(gan_data_1[n])
        ax[2, 0].set_title('Generated Style 1')
        ax[2, 0].set_axis_off()

        ax[0, 1].imshow(do_canny_single(openAI_data[n], None, "OpenAI"), cmap=cm.gray)
        ax[0, 1].set_title('Input OpenAI image sample')
        ax[0, 1].set_axis_off()

        ax[1, 1].imshow(do_canny_single(gan_data_0[n], 0, "real"), cmap=cm.gray)
        ax[1, 1].set_title('Generated Style 0')
        ax[1, 1].set_axis_off()

        ax[2, 1].imshow(do_canny_single(gan_data_1[n], 1, "real"), cmap=cm.gray)
        ax[2, 1].set_title('Generated Style 1')
        ax[2, 1].set_axis_off()
        
        
        l = 0 #loss
        for l in range(3):
            ax[l, 2].cla()
            ax[l, 2].set_title(losses_n[l])
            ax[l, 2].scatter(n, losses_0[n,l], color='g')
            ax[l, 2].scatter(n, losses_1[n,l], color='brown')
            ax[l, 2].plot(X, losses_0[:,l], color='g')
            ax[l, 2].plot(X, losses_1[:,l], color='brown')
            ax[l, 2].bar(-5, losses_0[:,l].mean(), color='g', label="Style 0")
            ax[l, 2].bar(-3, losses_1[:,l].mean(), color='brown', label="Style 1")
            ax[l, 2].set_ylim(np.min([losses_0[:,l],losses_1[:,l]])* 0.99, np.max([losses_0[:,l],losses_1[:,l]])*1.01)
            ax[l, 2].set_xlabel("Frame")
            ax[l, 2].set_ylabel(losses_n[l])
            ax[l ,2].legend()
            
        for l in range(3,6):
            ax[l-3, 3].cla()
            ax[l-3, 3].set_title(losses_n[l])
            ax[l-3, 3].scatter(n, losses_0[n,l], color='g')
            ax[l-3, 3].scatter(n, losses_1[n,l], color='brown')
            ax[l-3, 3].plot(X, losses_0[:,l], color='g')
            ax[l-3, 3].plot(X, losses_1[:,l], color='brown')
            ax[l-3, 3].bar(-5, losses_0[:,l].mean(), color='g', label="Style 0")
            ax[l-3, 3].bar(-3, losses_1[:,l].mean(), color='brown', label="Style 1")
            ax[l-3, 3].set_ylim(np.min([losses_0[:,l],losses_1[:,l]])* 0.99, np.max([losses_0[:,l],losses_1[:,l]])*1.01)
            ax[l-3, 3].set_xlabel("Frame")
            ax[l-3, 3].set_ylabel(losses_n[l])
            ax[l-3, 3].legend()

        
        plt.tight_layout()
    
    if save:
        steps = 5
    else:
        steps=1
    anim = FuncAnimation(fig, update, frames=np.arange(0,OpenAI_edges.shape[0], steps), interval=500, save_count=sys.maxsize)
    if save:
        anim.save('Slides/edges_complete.gif', dpi=150, writer='imagemagick')
        print(f'Slides/edges_complete.gif done')

    return anim

In [ ]:
def edge_comp_box_plots(experiments, OpenAI_edges, gan_data_0_edges, gan_data_1_edges, save=False):
    openAI_data, gan_data_0, gan_data_1 = experiments
    n = 0
    X = range(OpenAI_edges.shape[0])
    fig, ax = plt.subplots(3, 4, figsize=(20, 10))
    fig.patch.set_alpha(1.)

    
    losses_0 = np.array([get_losses(y, x) for (x,y) in  zip(gan_data_0_edges,OpenAI_edges)])
    losses_1 = np.array([get_losses(y, x) for (x,y) in  zip(gan_data_1_edges,OpenAI_edges)])
    losses_n = [r"Accuracy $\uparrow$", r"MSE $\downarrow$", r"BCE $\downarrow$", r"SSIM $\uparrow$", r"PSNR $\uparrow$", r"Cosine similarity"]

    def update(n):
        fig.suptitle(f"Structure comparison, frame {n}")
        ax[0, 0].imshow(openAI_data[n])
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()

        ax[1, 0].imshow(gan_data_0[n])
        ax[1, 0].set_title('Generated Style 0')
        ax[1, 0].set_axis_off()

        ax[2, 0].imshow(gan_data_1[n])
        ax[2, 0].set_title('Generated Style 1')
        ax[2, 0].set_axis_off()

        ax[0, 1].imshow(do_canny_single(openAI_data[n], None, "OpenAI"), cmap=cm.gray)
        ax[0, 1].set_title('Input OpenAI image sample')
        ax[0, 1].set_axis_off()

        ax[1, 1].imshow(do_canny_single(gan_data_0[n], 0, "real"), cmap=cm.gray)
        ax[1, 1].set_title('Generated Style 0')
        ax[1, 1].set_axis_off()

        ax[2, 1].imshow(do_canny_single(gan_data_1[n], 1, "real"), cmap=cm.gray)
        ax[2, 1].set_title('Generated Style 1')
        ax[2, 1].set_axis_off()
        
        
        l = 0 #loss
        for c in [2,3]:
            for r in range(3):
                ax[r, c].cla()
                ax[r, c].set_title(losses_n[l])
                ax[r, c].scatter([0,1], [losses_0[n,l],losses_1[n,l]], color=['g','brown'])
                ax[r, c].boxplot([losses_0[:,l], losses_1[:,l]], positions = [0,1])
                ax[r, c].set_xticks([0, 1])
                ax[r, c].set_xticklabels(["style 0", "style 1"])
                ax[r, c].set_ylabel(losses_n[l])
                l+=1

        
        plt.tight_layout()
   

    if save:
        steps = 5
    else:
        steps=1
        
    anim = FuncAnimation(fig, update, frames=np.arange(0,OpenAI_edges.shape[0], steps), interval=500, save_count=sys.maxsize)    
    
    if save:
        anim.save('Slides/edges_complete_boxplot.gif', dpi=150, writer='imagemagick')
        print(f'Slides/edges_complete_boxplot.gif done')
        
    return anim

In [ ]:
def edge_only_box_plots(OpenAI_edges,edges,edge_names,eval_type,save_steps=10, save=False):
    n = 0
    X = range(OpenAI_edges.shape[0])
    fig, ax = plt.subplots(3, 5, figsize=(25, 10))
    fig.patch.set_alpha(1.)
    
    losses = np.array([np.array([get_losses(y, x) for (x,y) in zip(edge,OpenAI_edges)]) for edge in edges])
    losses_n = [r"Accuracy $\uparrow$", r"MSE $\downarrow$", r"BCE $\downarrow$", r"SSIM $\uparrow$", r"PSNR $\uparrow$", r"Cosine similarity"]
    local_names=['net A_0', 'net A_1', 'net B_0','net B_1','net C_0','net C_1']
    
    fig.suptitle(f"Structure comparison, frame {n}")
    ax[0, 0].imshow(openAI_data[n])
    ax[0, 0].set_title('Input OpenAI image sample')
    ax[0, 0].set_axis_off()

    ax[0, 1].imshow(do_canny_single(openAI_data[n], None, "OpenAI"), cmap=cm.gray)
    ax[0, 1].set_title('Input OpenAI image sample')
    ax[0, 1].set_axis_off()


    ax[0, 2].set_axis_off()


    ax[1, 0].imshow(edges[0][n].astype(bool), cmap=cm.gray)
    ax[1, 0].set_title(f'Generated {edge_names[0]},\n {local_names[0]}')
    ax[1, 0].set_axis_off()

    ax[2, 0].imshow(edges[1][n].astype(bool), cmap=cm.gray)
    ax[2, 0].set_title(f'Generated {edge_names[1]},\n {local_names[1]}')
    ax[2, 0].set_axis_off()

    ax[1, 1].imshow(edges[2][n].astype(bool), cmap=cm.gray)
    ax[1, 1].set_title(f'Generated {edge_names[2]},\n {local_names[2]}')
    ax[1, 1].set_axis_off()

    ax[2, 1].imshow(edges[3][n].astype(bool), cmap=cm.gray)
    ax[2, 1].set_title(f'Generated {edge_names[3]},\n {local_names[3]}')
    ax[2, 1].set_axis_off()

    ax[1, 2].imshow(edges[4][n].astype(bool), cmap=cm.gray)
    ax[1, 2].set_title(f'Generated {edge_names[4]},\n {local_names[4]}')
    ax[1, 2].set_axis_off()

    ax[2, 2].imshow(edges[5][n].astype(bool), cmap=cm.gray)
    ax[2, 2].set_title(f'Generated {edge_names[5]},\n {local_names[5]}')
    ax[2, 2].set_axis_off()
        

        
    l = 0 #loss
    for c in [3,4]:
        for r in range(3):
            ax[r, c].cla()
            ax[r, c].set_title(losses_n[l])
            ax[r, c].scatter(range(losses.shape[0]), [loss[n,l] for loss in losses], color=['g','brown']*(losses.shape[0]//2))
            ax[r, c].boxplot([loss[:,l] for loss in losses], positions = range(losses.shape[0]))
            ax[r, c].set_xticks(range(losses.shape[0]))
            ax[r, c].set_xticklabels(local_names)
            ax[r, c].set_ylabel(losses_n[l])
            l+=1


    plt.tight_layout()

    def update(n):
        fig.suptitle(f"Structure comparison, frame {n}")
        ax[0, 0].imshow(openAI_data[n])
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()

        ax[0, 1].imshow(do_canny_single(openAI_data[n], None, "OpenAI"), cmap=cm.gray)
        ax[0, 1].set_title('Input OpenAI image sample')
        ax[0, 1].set_axis_off()
        

        ax[0, 2].set_axis_off()

        
        ax[1, 0].imshow(edges[0][n].astype(bool), cmap=cm.gray)
        ax[1, 0].set_title(f'Generated {edge_names[0]},\n {local_names[0]}')
        ax[1, 0].set_axis_off()

        ax[2, 0].imshow(edges[1][n].astype(bool), cmap=cm.gray)
        ax[2, 0].set_title(f'Generated {edge_names[1]},\n {local_names[1]}')
        ax[2, 0].set_axis_off()
        
        ax[1, 1].imshow(edges[2][n].astype(bool), cmap=cm.gray)
        ax[1, 1].set_title(f'Generated {edge_names[2]},\n {local_names[2]}')
        ax[1, 1].set_axis_off()

        ax[2, 1].imshow(edges[3][n].astype(bool), cmap=cm.gray)
        ax[2, 1].set_title(f'Generated {edge_names[3]},\n {local_names[3]}')
        ax[2, 1].set_axis_off()
        
        ax[1, 2].imshow(edges[4][n].astype(bool), cmap=cm.gray)
        ax[1, 2].set_title(f'Generated {edge_names[4]},\n {local_names[4]}')
        ax[1, 2].set_axis_off()

        ax[2, 2].imshow(edges[5][n].astype(bool), cmap=cm.gray)
        ax[2, 2].set_title(f'Generated {edge_names[5]},\n {local_names[5]}')
        ax[2, 2].set_axis_off()
        
        
        
        l = 0 #loss
        for c in [3,4]:
            for r in range(3):
                ax[r, c].cla()
                ax[r, c].set_title(losses_n[l])
                ax[r, c].scatter(range(losses.shape[0]), [loss[n,l] for loss in losses], color=['g','brown']*(losses.shape[0]//2))
                ax[r, c].boxplot([loss[:,l] for loss in losses], positions = range(losses.shape[0]))
                ax[r, c].set_xticks(range(losses.shape[0]))
                ax[r, c].set_xticklabels(local_names)
                ax[r, c].set_ylabel(losses_n[l])
                l+=1

        
        #plt.tight_layout()
   

    if save:
        steps = save_steps
    else:
        steps=5
        
    anim = FuncAnimation(fig, update, frames=np.arange(0,OpenAI_edges.shape[0]-1, steps), interval=2000, save_count=sys.maxsize)    
    
    if save:        
        anim.save(f'Slides/edges_only_boxplot_{eval_type}_{save_steps}.gif', dpi=150, writer='imagemagick')
        print(f'Slides/edges_only_boxplot_{eval_type}_{save_steps}.gif done')
        
    return anim

In [ ]:
def style_transfer_only(experiments,names,eval_type,save_steps=10, save=False):
    OpenAI = experiments[0]
    names= names[1:]
    experiments = experiments[1:]
    n = 0
    fig, ax = plt.subplots(3, 7, figsize=(25, 10))
    fig.patch.set_alpha(1.)
    
    local_names=['net A_0', 'net A_1', 'net B_0','net B_1','net C_0','net C_1','net D_0', 'net D_1', 'net E_0','net E_1','net F_0','net F_1','net G_0','net G_1']
    long_names=[n[10:-2] for n in names[::2]]
    name_conv='\n'.join([rf"{long} $\rightarrow$ {local[:-2]}" for (long,local) in zip(long_names,local_names[::2])])
    
    
    
    fig.suptitle(f"Style transfer comparison, frame {n} \n {name_conv} ")
    ax[0, 0].imshow(OpenAI[n])
    ax[0, 0].set_title('Input OpenAI image sample')
    ax[0, 0].set_axis_off()
    ax[0, 1].set_axis_off()
    ax[0, 2].set_axis_off()
    ax[0, 3].set_axis_off()
    ax[0, 4].set_axis_off()
    ax[0, 5].set_axis_off()
    ax[0, 6].set_axis_off()

    index =0
    for i in range(names.shape[0]//2):


        ax[1, i].imshow(experiments[index][n])
        ax[1, i].set_title(local_names[index])
        ax[1, i].set_axis_off()

        ax[2, i].imshow(experiments[index+1][n])
        ax[2, i].set_title(local_names[index+1])
        ax[2, i].set_axis_off()

        index +=2

    plt.tight_layout()
    
    def update(n):
        fig.suptitle(f"Style transfer comparison, frame {n} \n {name_conv} ")
        ax[0, 0].imshow(OpenAI[n])
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()
        ax[0, 1].set_axis_off()
        ax[0, 2].set_axis_off()
        ax[0, 3].set_axis_off()
        ax[0, 4].set_axis_off()
        ax[0, 5].set_axis_off()
        ax[0, 6].set_axis_off()
    
        index =0
        for i in range(names.shape[0]//2):
            
            
            ax[1, i].imshow(experiments[index][n])
            ax[1, i].set_title(local_names[index])
            ax[1, i].set_axis_off()
            
            ax[2, i].imshow(experiments[index+1][n])
            ax[2, i].set_title(local_names[index+1])
            ax[2, i].set_axis_off()
        
            index +=2
        
        #plt.tight_layout()
   

    if save:
        steps = save_steps
    else:
        steps=5
        
    anim = FuncAnimation(fig, update, frames=np.arange(0,OpenAI.shape[0]-1, steps), interval=2000, save_count=sys.maxsize)    
    
    if save:
        anim.save(f'Slides/style_transfer_only_{eval_type}_{save_steps}.gif', dpi=150, writer='imagemagick')
        print(f'Slides/style_transfer_only_{eval_type}_{save_steps}.gif done')
        
    return anim

In [ ]:
def canny_only(experiments,names,eval_type,steps, step_skip=5, save=False):
    OpenAI = experiments[0]
    names= names[1:]
    experiments = experiments[1:]
    n = 0
    fig, ax = plt.subplots(3, 7, figsize=(35, 25))
    fig.patch.set_alpha(1.)
    
    local_names=['net A_0', 'net A_1', 'net B_0','net B_1','net C_0','net C_1','net D_0', 'net D_1', 'net E_0','net E_1','net F_0','net F_1','net G_0','net G_1']
    long_names=[n[10:-2] for n in names[::2]]
    name_conv='\n'.join([rf"{long} $\rightarrow$ {local[:-2]}" for (long,local) in zip(long_names,local_names[::2])])
    
    
    
    fig.suptitle(f"Style transfer comparison, frame {n} ")
    ax[0, 0].imshow(OpenAI[n].astype(bool), cmap=cm.gray)
    ax[0, 0].set_title('Input OpenAI image sample')
    ax[0, 0].set_axis_off()
    ax[0, 1].set_axis_off()
    ax[0, 2].set_axis_off()
    ax[0, 3].set_axis_off()
    ax[0, 4].set_axis_off()
    ax[0, 5].set_axis_off()
    ax[0, 6].set_axis_off()

    index =0
    for i in range(names.shape[0]//2):


        ax[1, i].imshow(experiments[index][n].astype(bool), cmap=cm.gray)
        ax[1, i].set_title(local_names[index])
        ax[1, i].set_axis_off()

        ax[2, i].imshow(experiments[index+1][n].astype(bool), cmap=cm.gray)
        ax[2, i].set_title(local_names[index+1])
        ax[2, i].set_axis_off()

        index +=2

    plt.tight_layout()
    
    def update(n):
        fig.suptitle(f"Style transfer comparison, frame {n} ")
        ax[0, 0].imshow(OpenAI[n].astype(bool), cmap=cm.gray)
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()
        ax[0, 1].set_axis_off()
        ax[0, 2].set_axis_off()
        ax[0, 3].set_axis_off()
        ax[0, 4].set_axis_off()
        ax[0, 5].set_axis_off()
        ax[0, 6].set_axis_off()
    
        index =0
        for i in range(names.shape[0]//2):
            
            
            ax[1, i].imshow(experiments[index][n].astype(bool), cmap=cm.gray)
            ax[1, i].set_title(local_names[index])
            ax[1, i].set_axis_off()
            
            ax[2, i].imshow(experiments[index+1][n].astype(bool), cmap=cm.gray)
            ax[2, i].set_title(local_names[index+1])
            ax[2, i].set_axis_off()
        
            index +=2
        
        #plt.tight_layout()
   

    anim = FuncAnimation(fig, update, frames=np.arange(0,steps*step_skip, step_skip), interval=50, save_count=sys.maxsize)    
    
    if save:
        anim.save(f'Slides/canny_only{eval_type}_{steps}.mp4',  writer='imagemagick')
        print(f'Slides/canny_only{eval_type}_{steps}.mp4 done')
        
    return anim

In [ ]:
def style_transfer_with_car(experiments,names,eval_type,steps,step_skip=5, save=False):
    car = np.load("car_positions.npz")

    OpenAI = experiments[0]
    names= names[1:]
    experiments = experiments[1:]
    n = 0
    fig, ax = plt.subplots(3, 7, figsize=(35, 25))
    fig.patch.set_alpha(1.)
    
    local_names=['net A_0', 'net A_1', 'net B_0','net B_1','net C_0','net C_1','net D_0', 'net D_1', 'net E_0','net E_1','net F_0','net F_1','net G_0','net G_1']
    long_names=[n[10:-2] for n in names[::2]]
    name_conv='\n'.join([rf"{long} $\rightarrow$ {local[:-2]}" for (long,local) in zip(long_names,local_names[::2])])
    
    
    
    fig.suptitle(f"Style transfer comparison, frame {n}")
    frame = np.copy(OpenAI[n])
    frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
    frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
    
    ax[0, 0].imshow(frame)
    ax[0, 0].set_title('Input OpenAI image sample')
    ax[0, 0].set_axis_off()
    ax[0, 1].set_axis_off()
    ax[0, 2].set_axis_off()
    ax[0, 3].set_axis_off()
    ax[0, 4].set_axis_off()
    ax[0, 5].set_axis_off()
    ax[0, 6].set_axis_off()

    index =0
    for i in range(names.shape[0]//2):

        
        frame = np.copy(experiments[index][n])
        frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
        frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
        ax[1, i].imshow(frame)
        ax[1, i].set_title(local_names[index])
        ax[1, i].set_axis_off()
        
        frame = np.copy(experiments[index+1][n])
        frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
        frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
        ax[2, i].imshow(frame)
        ax[2, i].set_title(local_names[index+1])
        ax[2, i].set_axis_off()

        index +=2

    plt.tight_layout()
    
    def update(n):
        fig.suptitle(f"Style transfer comparison, frame {n} ")
        frame = np.copy(OpenAI[n])
        frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
        frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]

        ax[0, 0].imshow(frame)
        ax[0, 0].set_title('Input OpenAI image sample')
        ax[0, 0].set_axis_off()
        ax[0, 1].set_axis_off()
        ax[0, 2].set_axis_off()
        ax[0, 3].set_axis_off()
        ax[0, 4].set_axis_off()
        ax[0, 5].set_axis_off()
        ax[0, 6].set_axis_off()
    
        index =0
        for i in range(names.shape[0]//2):

            frame = np.copy(experiments[index][n])
            frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
            frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
            ax[1, i].imshow(frame)
            ax[1, i].set_title(local_names[index])
            ax[1, i].set_axis_off()

            frame = np.copy(experiments[index+1][n])
            frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
            frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
            ax[2, i].imshow(frame)
            ax[2, i].set_title(local_names[index+1])
            ax[2, i].set_axis_off()
        
            index +=2
        
        #plt.tight_layout()
   


        
    anim = FuncAnimation(fig, update, frames=np.arange(0,steps*step_skip, step_skip), interval=50, save_count=sys.maxsize)    
    
    if save:
        anim.save(f'Slides/style_transfer_with_car_{eval_type}_{steps}.mp4', writer='imagemagick')
        print(f'Slides/style_transfer_with_car_{eval_type}_{steps}.mp4 done')
        
    return anim

In [ ]:
#Imagemagick movie
style_transfer_car=style_transfer_with_car(experiments,names,eval_type,steps=200, step_skip=5, save=True)
#HTML(style_transfer_car.to_jshtml())

In [ ]:
cany_on=canny_only(generated_edges,names,eval_type,steps=200, step_skip=5, save=True)
#HTML(cany_on.to_jshtml())


In [ ]:
style_transfer_only=style_transfer_only(experiments,names,eval_type,save_steps=50, save=True)
#HTML(style_transfer_only.to_jshtml())

In [ ]:
edges_box=edge_only_box_plots(OpenAI_edges,edges,edge_names,eval_type,save_steps=50, save=True)
#HTML(edges_box.to_jshtml())

In [ ]:
def metrics_only(edges,names,eval_type, save=False):

    OpenAI = edges[0]
    names= names[1:]
    experiments = edges[1:]
    n = 0
    
    X = range(OpenAI.shape[0])

    fig, ax = plt.subplots(3, 2, figsize=(25, 15))
    fig.patch.set_alpha(1.)
    
    local_names=['net A_0', 'net A_1', 'net B_0','net B_1','net C_0','net C_1','net D_0', 'net D_1', 'net E_0','net E_1','net F_0','net F_1','net G_0','net G_1']
    local_names = local_names[::2]+local_names[1::2]

    losses = [np.array([get_losses(y, x) for (x,y) in  zip(exp,OpenAI_edges)]) for exp in experiments]
    losses_n = [r"Accuracy $\uparrow$", r"MSE $\downarrow$", r"BCE $\downarrow$", r"SSIM $\uparrow$", r"PSNR $\uparrow$", r"Cosine similarity"]



    fig.suptitle(f"Structure comparison")
    
        
    l = 0 #loss
    for c in [0,1]:
        for r in range(3):
            ax[r, c].cla()
            ax[r, c].set_title(losses_n[l])
            ax[r, c].boxplot([loss[:,l] for loss in losses][::2], positions = range(len(losses)//2))
            ax[r, c].boxplot([loss[:,l] for loss in losses][1::2], positions = range(len(losses)//2,len(losses)))
            ax[r, c].set_xticks(range(len(losses)))
            ax[r, c].set_xticklabels(local_names)
            ax[r, c].set_ylabel(losses_n[l])
            l+=1

    #plt.tight_layout()


    if save:
        fig.savefig(f'Slides/metrics_only_{eval_type}.png', dpi=300)
        print(f'Slides/metrics_only_{eval_type}.mp4 done')
    
    plt.show()
        


In [ ]:
metrics_only= metrics_only(generated_edges,names,eval_type, save=True)


In [ ]:
    ax[0, 4] = histo_single(real_data.reshape(real_data.shape[0] * real_data.shape[1], real_data.shape[2], real_data.shape[3]), f"mean Real", ax[0, 4])


In [ ]:
real_style_0.reshape(real_style_0.shape[0] * real_style_0.shape[1], real_style_0.shape[2], real_style_0.shape[3]).shape

In [ ]:
real_style_0.shape

In [ ]:
[ex.reshape(ex.shape[0] * ex.shape[1], ex.shape[2], ex.shape[3]).shape for ex in experiments[1::2]]

In [ ]:
experiments.shape

In [ ]:
def histo_single_grayscale(image, title, ax):
    img = [image]
    img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in img])
    hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256])
    ax.cla()
    ax.set_title(f"Grayscale Histogram {title}")
    ax.set_xlabel("Bins")
    ax.set_ylabel("# of Pixels")
    ax.plot(hist)
    ax.set_xlim([0, 256])
    return ax


def histo_single(image, title, ax):
    img = [image]
    colors = ("r", "g", "b")
    ax.cla()
    ax.set_title(f"'Flattened' Color Histogram {title}")
    ax.set_xlabel("Bins")
    ax.set_ylabel("# of Pixels")
    for channel in range(3):
        hist = cv2.calcHist(img, [channel], None, [256], [0, 256])
        ax.plot(hist, color=colors[channel])
        ax.set_xlim([0, 256])
    return ax


def real_vs_gan_histo(real_data, gan_data, style, save=False):
    fig, ax = plt.subplots(2, 5, figsize=(30, 10))
    fig.patch.set_alpha(1.)

    n = 0
    fig.suptitle(f"Histogram comparison for style {style}, frame {n}")
    ax[0, 0].imshow(real_data[n])
    ax[0, 0].set_title('Input real image sample')
    ax[0, 0].set_axis_off()

    ax[0, 1] = histo_single_grayscale(real_data[n], f"single image Real", ax[0, 1])
    ax[0, 2] = histo_single_grayscale(real_data.reshape(real_data.shape[0] * real_data.shape[1], real_data.shape[2], real_data.shape[3]), f"mean Real", ax[0, 2])
    ax[0, 3] = histo_single(real_data[n], f"single image Real", ax[0, 3])
    ax[0, 4] = histo_single(real_data.reshape(real_data.shape[0] * real_data.shape[1], real_data.shape[2], real_data.shape[3]), f"mean Real", ax[0, 4])

    ax[1, 0].imshow(gan_data[n])
    ax[1, 0].set_title('Generated image sample')
    ax[1, 0].set_axis_off()

    ax[1, 1] = histo_single_grayscale(gan_data[n], f"single image Generated", ax[1, 1])
    ax[1, 2] = histo_single_grayscale(gan_data.reshape(gan_data.shape[0] * gan_data.shape[1], gan_data.shape[2], gan_data.shape[3]), f"mean Generated", ax[1, 2])
    ax[1, 3] = histo_single(gan_data[n], f"single image Generated", ax[1, 3])
    ax[1, 4] = histo_single(gan_data.reshape(gan_data.shape[0] * gan_data.shape[1], gan_data.shape[2], gan_data.shape[3]), f"mean Generated", ax[1, 4])
    plt.tight_layout()

In [ ]:
def histo_comp(real_style_0,real_style_1,experiments,names, save=False):

    fig, ax = plt.subplots(2, 4, figsize=(25, 10))



    img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in [real_style_0.reshape(real_style_0.shape[0] * real_style_0.shape[1], real_style_0.shape[2], real_style_0.shape[3])]])
    target_hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256])
    ax[0,0].plot(target_hist/(real_style_0.shape[0] * real_style_0.shape[1]), c='b', label='target',linewidth=3.0)

    for i in range(1,experiments.shape[0],2):
        exp = experiments[i]
        img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in [exp.reshape(exp.shape[0] * exp.shape[1], exp.shape[2], exp.shape[3])]])
        target_hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256])
        ax[0,0].plot(target_hist/(exp.shape[0] * exp.shape[1]), label=names[i][10:])
    ax[0,0].legend()
    ax[0,0].set_title("Gray histogram Style 0 (Normalized over number of image)")


    img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in [real_style_1.reshape(real_style_1.shape[0] * real_style_1.shape[1], real_style_1.shape[2], real_style_1.shape[3])]])
    target_hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256])
    ax[1,0].plot(target_hist/(real_style_1.shape[0] * real_style_1.shape[1]), c='b', label='target',linewidth=3.0)

    for i in range(2,experiments.shape[0],2):
        exp = experiments[i]
        img_gray = np.array([cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in [exp.reshape(exp.shape[0] * exp.shape[1], exp.shape[2], exp.shape[3])]])
        target_hist = cv2.calcHist(img_gray, [0], None, [256], [0, 256], accumulate=True)
        ax[1,0].plot(target_hist/(exp.shape[0] * exp.shape[1]), label=names[i][10:])
    ax[1,0].legend()
    ax[1,0].set_title("Gray histogram Style 1 (Normalized over number of image)")



    col=["Red","Green","Blue"]
    for channel in range(3):


        target_hist = cv2.calcHist([real_style_0.reshape(real_style_0.shape[0] * real_style_0.shape[1], real_style_0.shape[2], real_style_0.shape[3])], [channel], None, [256], [0, 256])
        ax[0,channel+1].plot(target_hist/(real_style_0.shape[0] * real_style_0.shape[1]), c='b', label='target',linewidth=3.0)

        for i in range(1,experiments.shape[0],2):
            exp = experiments[i]
            target_hist = cv2.calcHist([exp.reshape(exp.shape[0] * exp.shape[1], exp.shape[2], exp.shape[3])], [channel], None, [256], [0, 256])
            ax[0,channel+1].plot(target_hist/(exp.shape[0] * exp.shape[1]), label=names[i][10:])
        ax[0,channel+1].legend()
        ax[0,channel+1].set_title(f"{col[channel]} channel histogram Style 0")



    col=["Red","Green","Blue"]
    for channel in range(3):


        target_hist = cv2.calcHist([real_style_1.reshape(real_style_1.shape[0] * real_style_1.shape[1], real_style_1.shape[2], real_style_1.shape[3])], [channel], None, [256], [0, 256])
        ax[1,channel+1].plot(target_hist/(real_style_1.shape[0] * real_style_1.shape[1]), c='b', label='target',linewidth=3.0)

        for i in range(2,experiments.shape[0],2):
            exp = experiments[i]
            target_hist = cv2.calcHist([exp.reshape(exp.shape[0] * exp.shape[1], exp.shape[2], exp.shape[3])], [channel], None, [256], [0, 256])
            ax[1,channel+1].plot(target_hist/(exp.shape[0] * exp.shape[1]), label=names[i][10:])
        ax[1,channel+1].legend()
        ax[1,channel+1].set_title(f"{col[channel]} channel histogram Style 1")

    plt.tight_layout()
    if save:
        fig.savefig(f'Slides/histo_comp_{eval_type}.png', dpi=300)
        print(f'Slides/histo_comp_{eval_type}.png done')
    
    plt.show()

In [ ]:
histo_comp(real_style_0,real_style_1,experiments,names, save=True)

# Curvature comparison
### Curve comparison with boxplots

In [ ]:
curve_box=edge_comp_box_plots(experiments, OpenAI_edges, gan_data_0_edges, gan_data_1_edges)#, save=True)
HTML(curve_box.to_jshtml())


### Curve comparisson with graphs

In [ ]:
curve_graph=edge_comp_graphs(experiments, OpenAI_edges, gan_data_0_edges, gan_data_1_edges)#, save=True)
HTML(curve_graph.to_jshtml())

### Contours comparison

In [ ]:
curve_dect=curve_detection(experiments[0], real_style_0, real_style_1)
HTML(curve_dect.to_jshtml())


# Histograms

In [ ]:
histo_0 = real_vs_gan_histo(real_style_0, experiments[1], "0")
HTML(histo_0.to_jshtml())


In [ ]:
histo_1 = real_vs_gan_histo(real_style_1, experiments[2], "1")
HTML(histo_1.to_jshtml())


In [ ]:
car = np.load("car_positions.npz")
width = 800
height = 800
OpenAI = experiments[0]
fig, ax = plt.subplots(1, 3, figsize=(35, 25))
fig.patch.set_alpha(1.)
ns = np.random.randint(experiments[0].shape[0],  size=3)

for i,n in enumerate(ns):
    frame = np.copy(OpenAI[n])
    frame[np.where(car['frame'][:, :, 0] == 204)] = [204, 0, 0]
    frame[np.where(car['frame'][:256, :, 0] == 0)] = [0, 0, 0]
    
    frame = cv2.resize(frame, dsize=(width, height), interpolation=cv2.INTER_AREA)
    ax[i].imshow(frame)
    ax[i].set_title('Input OpenAI image sample')
    ax[i].set_axis_off()

plt.tight_layout()

fig.savefig(f'Slides/OpenAI_examples_{eval_type}.png', dpi=600)

plt.show()

In [ ]:
np.ra